# Retrieval Chain

replicated from langchain, https://python.langchain.com/docs/get_started/quickstart#conversation-retrieval-chain

## initializing Openai

In [1]:
from openai import OpenAI
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

## loader

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [3]:
type(docs),len(docs)

(list, 1)

## Embeddings

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## Using local vector Store

In [7]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# splitting documents
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)


In [20]:
len(documents)

5

In [8]:
#keeping them as embeddings in a vector store, vector, with an index
vector = FAISS.from_documents(documents, embeddings)

In [9]:
type(vector)

langchain_community.vectorstores.faiss.FAISS

## Template including specific context and chain

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
type(document_chain)

langchain_core.runnables.base.RunnableBinding

In [21]:
# document_chain

## build retriever from vector store ,and add it to the chain

In [22]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [23]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

In [26]:
response.keys()

dict_keys(['input', 'context', 'answer'])

In [28]:
len(response['context'])

4

In [29]:
print(response['answer'])

LangSmith can help with testing by simplifying dataset uploading and providing tools to run chains over data points and visualize the outputs. The LangSmith client allows users to pull down a dataset and run a chain over them, logging the results to a new project associated with the dataset. Users can review the results, assign feedback to runs, and mark them as correct or incorrect directly in the web app. LangSmith also offers a set of evaluators that can be specified when initiating a test run, providing guidance on examples that should be looked at. Additionally, LangSmith facilitates human evaluation through annotation queues, allowing users to manually review and annotate runs for subjective qualities or to validate auto-evaluated runs.


## Conversational Retrieval chain

In [30]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

### dummy test

In [32]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response =  retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [38]:
len(response) # all are documents, so the retriever chain outputs 4 documents
# we will now ask out fo those 4 documents

4

In [34]:
response[0]

Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'})

### chaining document chain and retriver_chain

In [39]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [40]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response_combined_chain = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [42]:
type(response_combined_chain)

dict

In [43]:
response_combined_chain.keys()

dict_keys(['chat_history', 'input', 'context', 'answer'])

In [44]:
response_combined_chain['chat_history']

[HumanMessage(content='Can LangSmith help test my LLM applications?'),
 AIMessage(content='Yes!')]

In [47]:
# all the documents
# response_combined_chain['context'] 

In [48]:
response_combined_chain['input']

'Tell me how'

In [49]:
response_combined_chain['answer']

'LangSmith can help test your LLM applications in a few ways:\n\n1. Dataset Testing: Once you have a dataset, you can use LangSmith to run your LLM chain over the data points and visualize the outputs. This can be done using the LangSmith client, which makes it easy to pull down a dataset and run a chain over them. You can log the results to a new project associated with the dataset and review them. You can also assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.\n\n2. Evaluators: LangSmith provides a set of evaluators in the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. While these evaluators may not be perfect, they can guide your eye to examples that you should look at, especially when dealing with a large number of data points.\n\n3. Human Evaluation: LangSmith makes it easy to manually rev

In [57]:
len(retriever.get_relevant_documents(""))

4